# Precinct-wise Time Series Forecasts

In [8]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import prophet
import numpy as np
import pmdarima as pm
from sklearn.metrics import mean_squared_error

### Reading Data

In [9]:
# Due to it being in pieces, we'll first make a final dataset called `final_df`, 
# then we'll sum all `counts` over each month!

years = [i for i in range(2014,2024)]
dfs = []

for year in years:
    dfs.append(pd.read_csv(f"../../data/time_series/cleaned/TS_{year}.csv"))
    dfs[-1]['Issue Date'] = pd.to_datetime(dfs[-1]['Issue Date'])

In [10]:
final_df0 = pd.concat(dfs, axis=0)
final_df0 = final_df0.reset_index(drop = True)

In [11]:
# remove jan 2015 spike and covid dip (mar-july 2020)
final_df0 = final_df0.drop(final_df0[(final_df0['Issue Date'] >= '2015-01-01') & (final_df0['Issue Date'] <= '2015-02-01')].index)
final_df0 = final_df0.drop(final_df0[(final_df0['Issue Date'] >= '2020-03-01') & (final_df0['Issue Date'] <= '2020-07-01')].index)

In [12]:
display(final_df0)

,Issue Date,Violation Precinct,Counts
0,2013-07-02,1,0
1,2013-07-02,5,0
2,2013-07-02,6,0
3,2013-07-02,7,0
4,2013-07-02,9,0
...,...,...,...
279120,2023-06-30,115,8
279121,2023-06-30,120,1
279122,2023-06-30,121,0
279123,2023-06-30,122,1


In [13]:
precinct_list = precinct_list = [1, 5, 6, 7, 9, 10, 13, 14, 17, 18, 19, 
                                 20, 22, 23, 24, 25, 26, 28, 30, 32, 33, 
                                 34, 40, 41, 42, 43, 44, 45, 46, 47, 48, 
                                 49, 50, 52, 60, 61, 62, 63, 66, 67, 68, 
                                 69, 70, 71, 72, 76, 78, 73, 75, 77, 79, 
                                 81, 83, 84, 88, 90, 94, 100, 101, 102, 
                                 103, 105, 106, 107, 113, 104, 108, 109, 
                                 110, 111, 112, 114, 115, 120, 121, 122, 
                                 123]

### Repeat time series forecasting from "Prediction Values Part 2" for each precinct

In [14]:
arima_models = []

for precinct in precinct_list:

    print(f"Starting precinct {precinct}.")

    # filter for precinct
    final_df = final_df0[final_df0["Violation Precinct"] == precinct].reset_index(drop = True)
    final_df = final_df.drop(columns=["Violation Precinct"])

    # create time as index and then summing per month
    final_df.set_index('Issue Date', inplace=True)

    final_df_monthly = final_df.resample('MS').sum()

    #### FIT MODELS (use RMSE)

    #### ARIMA model
    # Load the data and split it into separate pieces (array form)
    data = np.array(final_df_monthly['Counts'])
    train, test = pm.model_selection.train_test_split(data, train_size=108)

    # Fit a simple auto_arima model
    modl = pm.auto_arima(train, start_p=1, start_q=1, start_P=1, start_Q=1,
                        max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True,
                        stepwise=True, suppress_warnings=True, D=10, max_D=10,
                        error_action='ignore')

    # Create predictions for the future, evaluate on test
    preds, conf_int = modl.predict(n_periods=test.shape[0], return_conf_int=True, freq = "MS")

    # Print the error:
    print("Test RMSE: %.3f" % np.sqrt(mean_squared_error(test, preds)))

    #### PROPHET model
    # Create Prophet Object
    m = prophet.Prophet()

    # Use Similar Splitting (but not in array form)
    train_prophet, test_prophet = pm.model_selection.train_test_split(final_df_monthly, train_size = 108)

    # Make column names into those required by PROPHET
    train_prophet.reset_index(inplace=True)
    train_prophet = train_prophet.rename(columns = {'Issue Date':'ds'})
    train_prophet['y'] = train_prophet['Counts']

    # Fit
    m.fit(train_prophet)

    # Make a future data frame for another year (12 months)
    future = m.make_future_dataframe(periods=12, include_history = False, freq = "MS")

    # Predict
    forecast = m.predict(future)

    # Calculate RMSE
    preds_prophet = np.array(forecast['yhat'])
    test_prophet = np.array(test_prophet['Counts'])
    # Print the error:
    print("Test RMSE: %.3f" % np.sqrt(mean_squared_error(test_prophet, preds_prophet)))



    # plot
    prediction_data = {'True_Values': test, 'ARIMA_Predictions': preds, 'PROPHET_Predictions': preds_prophet}
    pred_df = pd.DataFrame(prediction_data)
    # ax = pred_df.plot(title = f"Precinct {precinct}")
    # ax.get_figure().savefig(f"../../Data/time_series/test_plots_nocovid/precinct_{precinct}.jpg")

    
    
    #### Fitted and predicted values

    # ARIMA

    data_arima = np.array(final_df_monthly['Counts'])
    modl = pm.auto_arima(data_arima, start_p=1, start_q=1, start_P=1, start_Q=1,
                        max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True,
                        stepwise=True, suppress_warnings=True, D=10, max_D=10,
                        error_action='ignore')
    preds_arima = modl.predict(n_periods=12, return_conf_int=False, freq = "MS")
    fitted_arima = modl.fittedvalues()
    preds_arima
    fitted_arima
    preds_arima = np.concatenate((fitted_arima, preds_arima))

    arima_models.append(modl)

    # PROPHET

    m = prophet.Prophet()
    data_prophet = final_df_monthly
    data_prophet.reset_index(inplace=True)
    data_prophet = data_prophet.rename(columns = {'Issue Date':'ds'})
    data_prophet['y'] = final_df_monthly['Counts']
    m.fit(data_prophet)
    future = m.make_future_dataframe(periods=12, include_history = True, freq = "MS")
    forecast = m.predict(future)
    preds_prophet = forecast[["ds","yhat"]]

    future = m.make_future_dataframe(periods=12, include_history = True, freq = "MS")

    output = future
    output = output.rename(columns = {"ds":"time"})
    preds_prophet = preds_prophet.rename(columns = {"ds":"time", "yhat":"prophet_pred"})
    preds_prophet["arima_pred"] = preds_arima
    output = output.merge(preds_prophet, how = "outer", on = "time")
    prep_final_df = final_df_monthly
    prep_final_df = prep_final_df.rename(columns = {"Issue Date":"time"})
    output = output.merge(prep_final_df, how = "outer", on = "time")
    # output = output.drop(columns = ["level_0", "index"])
    output = output.rename(columns = {"Counts": "values"})

    output.to_csv(f"../../data/time_series/precinct_line_graphs_nocovid/line_graph_monthly_data_pre{precinct}.csv")


    print(f"Finished precinct {precinct}.")


Starting precinct 1.


15:36:02 - cmdstanpy - INFO - Chain [1] start processing
15:36:02 - cmdstanpy - INFO - Chain [1] done processing


Test RMSE: 65.790
Test RMSE: 54.339


15:36:03 - cmdstanpy - INFO - Chain [1] start processing
15:36:03 - cmdstanpy - INFO - Chain [1] done processing


Finished precinct 1.
Starting precinct 5.


...

Finished precinct 122.
Starting precinct 123.


15:38:12 - cmdstanpy - INFO - Chain [1] start processing
15:38:12 - cmdstanpy - INFO - Chain [1] done processing


Test RMSE: 4.668
Test RMSE: 3.715


15:38:13 - cmdstanpy - INFO - Chain [1] start processing
15:38:13 - cmdstanpy - INFO - Chain [1] done processing


Finished precinct 123.


In [15]:
for model in arima_models:
    print(model)

 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(0,1,4)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(0,1,0)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(0,0,1)(0,0,0)[0] intercept
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(2,0,1)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(2,0,0)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(4,0,0)(0,0,0)[0] intercept
 ARIMA(2,1,1)(0,0,0)[0]          
 ARIMA(1,1,0)(0,0,0)[0]          
 ARIMA(3,1,1)(0,0,0)[0]          
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(0,0,0)[0] intercept
 ARIMA(1,1,1)(0,0,0)[0]          
 ARIMA(1,0,0)(